In [3]:
import pandas as pd
import re
from collections import defaultdict, Counter
import spacy
import glob
import string
from spacy.matcher import Matcher 
from spacy.training import Example
import os
import sys
import numpy as np
from pathlib import Path
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from rake_nltk import Rake
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')
from sklearn_crfsuite import CRF,metrics
import spacy
from collections import defaultdict, Counter
import networkx as nx
from spacy.matcher import PhraseMatcher
from collections import defaultdict
from tqdm.auto import tqdm
import string

In [4]:
folder_path = "archive/"
libros = {}
for filename in sorted(os.listdir(folder_path)):
    if filename.endswith(".txt"):
        with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as f:
            libros[filename] = f.read()
for nombre, texto in libros.items():
    print(f"{nombre}: {len(texto)} caracteres")


01 Harry Potter and the Sorcerers Stone.txt: 439516 caracteres
02 Harry Potter and the Chamber of Secrets.txt: 492336 caracteres
03 Harry Potter and the Prisoner of Azkaban.txt: 625764 caracteres
04 Harry Potter and the Goblet of Fire.txt: 1100552 caracteres
05 Harry Potter and the Order of the Phoenix.txt: 1499693 caracteres
06 Harry Potter and the Half-Blood Prince.txt: 988173 caracteres
07 Harry Potter and the Deathly Hallows.txt: 1140043 caracteres


In [3]:
#print(libros["01 Harry Potter and the Sorcerers Stone.txt"][:1000])  

In [4]:
for nombre_libro, texto_completo in list(libros.items())[5:6]:  # probar libro num
    print(f"\n=== DIAGNÓSTICO: {nombre_libro} ===")
    
    patron_mejorado = r'CHAPTER\s+[A-Z-]+(?:\w*)?'
    
    chapter_matches = []
    for match in re.finditer(patron_mejorado, texto_completo):
        start = match.start()
        end = start + 200  # Los siguientes 200 caracteres
        sample = texto_completo[start:end].replace('\n', '\\n')
        chapter_matches.append((match.group(), sample))
    
    print(f"Capítulos encontrados: {len(chapter_matches)}")
    for i, (chapter, sample) in enumerate(chapter_matches):
        print(f"{i+1}. {chapter}")
        #print(f"   Contexto: {sample}...")
    print()



=== DIAGNÓSTICO: 06 Harry Potter and the Half-Blood Prince.txt ===
Capítulos encontrados: 30
1. CHAPTER ONE
2. CHAPTER TWO
3. CHAPTER THREE
4. CHAPTER FOUR
5. CHAPTER FIVE
6. CHAPTER SIX
7. CHAPTER SEVEN
8. CHAPTER EIGHT
9. CHAPTER NINE
10. CHAPTER TEN
11. CHAPTER ELEVEN
12. CHAPTER TWELVE
13. CHAPTER THIRTEEN
14. CHAPTER FOURTEEN
15. CHAPTER FIFTEEN
16. CHAPTER SIXTEEN
17. CHAPTER SEVENTEEN
18. CHAPTER EIGHTEEN
19. CHAPTER NINETEEN
20. CHAPTER TWENTY
21. CHAPTER TWENTY-ONE
22. CHAPTER TWENTY-TWO
23. CHAPTER TWENTY-THREE
24. CHAPTER TWENTY-FOUR
25. CHAPTER TWENTY-FIVE
26. CHAPTER TWENTY-SIX
27. CHAPTER TWENTY-SEVEN
28. CHAPTER TWENTY-EIGTH
29. CHAPTER TWENTY-NINE
30. CHAPTER THRITY



In [5]:

# Preprocesamiento: eliminar líneas en blanco excesivas de los libros
libros_limpios = {}
for nombre, texto in libros.items():
    # Eliminar líneas completamente vacías y normalizar espacios
    lineas = texto.split('\n')
    lineas_limpias = []
    for linea in lineas:
        linea_limpia = linea.strip()
        if linea_limpia:  # Solo agregar líneas que no estén vacías
            lineas_limpias.append(linea_limpia)
    
    libros_limpios[nombre] = '\n'.join(lineas_limpias)
    print(f"{nombre}: {len(texto)} -> {len(libros_limpios[nombre])} caracteres (limpio)")

01 Harry Potter and the Sorcerers Stone.txt: 439516 -> 436029 caracteres (limpio)
02 Harry Potter and the Chamber of Secrets.txt: 492336 -> 488802 caracteres (limpio)
03 Harry Potter and the Prisoner of Azkaban.txt: 625764 -> 621157 caracteres (limpio)
04 Harry Potter and the Goblet of Fire.txt: 1100552 -> 1093698 caracteres (limpio)
05 Harry Potter and the Order of the Phoenix.txt: 1499693 -> 1489930 caracteres (limpio)
06 Harry Potter and the Half-Blood Prince.txt: 988173 -> 982205 caracteres (limpio)
07 Harry Potter and the Deathly Hallows.txt: 1140043 -> 1133088 caracteres (limpio)


In [6]:
# Procesamiento principal con regex mejorado
datos_estructurados = []
for nombre_libro, texto_completo in libros_limpios.items():
    print(f"Procesando: {nombre_libro}")
    
    
    patron_capitulos = r'CHAPTER\s+(?:[A-Z-]+(?:\w*)?|\d+)'
    
    # Buscar todas las posiciones donde aparece "CHAPTER" (case-insensitive)
    chapter_positions = []
    for match in re.finditer(patron_capitulos, texto_completo):
        chapter_positions.append(match.start())
    
    print(f"  - Capítulos encontrados: {len(chapter_positions)}")
    
    numero_capitulo = 1
    for i, pos in enumerate(chapter_positions):
        # Determinar el final de este capítulo (inicio del siguiente o final del texto)
        if i + 1 < len(chapter_positions):
            fin_capitulo = chapter_positions[i + 1]
        else:
            fin_capitulo = len(texto_completo)
        
        # Extraer el texto del capítulo completo
        texto_capitulo_completo = texto_completo[pos:fin_capitulo].strip()
        lineas_capitulo = texto_capitulo_completo.split('\n')
        
        if len(lineas_capitulo) >= 2:
            chapter_line = lineas_capitulo[0].strip()  # "CHAPTER THIRTY-SEVEN"
            titulo_capitulo = lineas_capitulo[1].strip()  # "THE BEGINNING"
            
            # El texto del capítulo es todo lo que sigue después del título
            texto_contenido = '\n'.join(lineas_capitulo[2:]).strip()
            
            # Solo procesar si el texto tiene contenido significativo
            if len(texto_contenido) > 100:
                # Limpiar texto básico
                texto_limpio = re.sub(r'\n+', ' ', texto_contenido)
                texto_limpio = re.sub(r'\s+', ' ', texto_limpio).strip()
                
                datos_estructurados.append({
                    'libro': nombre_libro.replace('.txt', ''),
                    'capitulo_num': numero_capitulo,
                    'chapter_line': chapter_line,  # "CHAPTER THIRTY-SEVEN"
                    'titulo_capitulo': titulo_capitulo,  # "THE BEGINNING"
                    'texto': texto_limpio,
                    'longitud': len(texto_limpio)
                })
                numero_capitulo += 1
                print(f"    Cap {numero_capitulo-1}: {chapter_line} - {titulo_capitulo[:50]}...")

Procesando: 01 Harry Potter and the Sorcerers Stone.txt
  - Capítulos encontrados: 17
    Cap 1: CHAPTER ONE - THE BOY WHO LIVED...
    Cap 2: CHAPTER TWO - THE VANISHING GLASS...
    Cap 3: CHAPTER THREE - LETTERS FROM NO ONE...
    Cap 4: CHAPTER FOUR - THE KEEPER OF THE KEYS...
    Cap 5: CHAPTER FIVE - DIAGON ALLEY...
    Cap 6: CHAPTER SIX - THE JOURNEY FROM PLATFORM NINE AND THREE-QUARTERS...
    Cap 7: CHAPTER SEVEN - THE SORTING HAT...
    Cap 8: CHAPTER EIGHT - THE POTIONS MASTER...
    Cap 9: CHAPTER NINE - THE MIDNIGHT DUEL...
    Cap 10: CHAPTER TEN - HALLOWEEN...
    Cap 11: CHAPTER ELEVEN - QUIDDITCH...
    Cap 12: CHAPTER TWELVE - THE MIRROR OF ERISED...
    Cap 13: CHAPTER THIRTEEN - NICHOLAS FLAMEL...
    Cap 14: CHAPTER FOURTEEN - NORBERT THE NORWEGIAN RIDGEBACK...
    Cap 15: CHAPTER FIFTEEN - THE FORBIDDEN FOREST...
    Cap 16: CHAPTER SIXTEEN - THROUGH THE TRAPDOOR...
    Cap 17: CHAPTER SEVENTEEN - THE MAN WITH TWO FACES...
Procesando: 02 Harry Potter and the Cham

In [7]:

print(f"\nDataset creado: {len(datos_estructurados)} capítulos en total")
df_libros = pd.DataFrame(datos_estructurados)
print(f"Libros procesados: {df_libros['libro'].nunique()}")
print(f"Distribución por libro:")
print(df_libros.groupby('libro').size())
df_libros.head()


Dataset creado: 197 capítulos en total
Libros procesados: 7
Distribución por libro:
libro
01 Harry Potter and the Sorcerers Stone         17
02 Harry Potter and the Chamber of Secrets      17
03 Harry Potter and the Prisoner of Azkaban     22
04 Harry Potter and the Goblet of Fire          37
05 Harry Potter and the Order of the Phoenix    38
06 Harry Potter and the Half-Blood Prince       30
07 Harry Potter and the Deathly Hallows         36
dtype: int64


,libro,capitulo_num,chapter_line,titulo_capitulo,texto,longitud
0,01 Harry Potter and the Sorcerers Stone,1,CHAPTER ONE,THE BOY WHO LIVED,"M r. and Mrs. Dursley, of number four, Privet ...",25725
1,01 Harry Potter and the Sorcerers Stone,2,CHAPTER TWO,THE VANISHING GLASS,N early ten years had passed since the Dursley...,19049
2,01 Harry Potter and the Sorcerers Stone,3,CHAPTER THREE,LETTERS FROM NO ONE,T he escape of the Brazilian boa constrictor e...,21432
3,01 Harry Potter and the Sorcerers Stone,4,CHAPTER FOUR,THE KEEPER OF THE KEYS,B OOM. They knocked again. Dudley jerked awake...,20229
4,01 Harry Potter and the Sorcerers Stone,5,CHAPTER FIVE,DIAGON ALLEY,H arry woke early the next morning. Although h...,37347


In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
nlp.max_length = 2_000_000  

In [1]:
# --- DICCIONARIOS DE ENTIDADES (CON CLAVES SIMPLES) ---
ENTIDADES_HP = {
    "PERSONAJES": {
    
    'harry': 'Harry Potter',
    'harry potter': 'Harry Potter',
    'potter': 'Harry Potter',
    
    'hermione': 'Hermione Granger',
    'hermione granger': 'Hermione Granger',
    'granger': 'Hermione Granger',
    
    'ron': 'Ron Weasley',
    'ron weasley': 'Ron Weasley',
    'ronald': 'Ron Weasley',
    'ronald weasley': 'Ron Weasley',
    'weasley': 'Ron Weasley',
    
    # Profesores y personal de Hogwarts
    'dumbledore': 'Albus Dumbledore',
    'albus dumbledore': 'Albus Dumbledore',
    'albus': 'Albus Dumbledore',
    'professor dumbledore': 'Albus Dumbledore',
    
    'mcgonagall': 'Minerva McGonagall',
    'minerva mcgonagall': 'Minerva McGonagall',
    'minerva': 'Minerva McGonagall',
    'professor mcgonagall': 'Minerva McGonagall',
    
    'hagrid': 'Rubeus Hagrid',
    'rubeus hagrid': 'Rubeus Hagrid',
    
    'snape': 'Severus Snape',
    'severus snape': 'Severus Snape',
    'severus': 'Severus Snape',
    'professor snape': 'Severus Snape',
    
    # Familia Dursley
    'vernon': 'Vernon Dursley',
    'vernon dursley': 'Vernon Dursley',
    'uncle vernon': 'Vernon Dursley',
    
    'petunia': 'Petunia Dursley',
    'petunia dursley': 'Petunia Dursley',
    'aunt petunia': 'Petunia Dursley',
    
    'dudley': 'Dudley Dursley',
    'dudley dursley': 'Dudley Dursley',
    
    # Villanos
    'voldemort': 'Lord Voldemort',
    'lord voldemort': 'Lord Voldemort',
    'you-know-who': 'Lord Voldemort',
    'he-who-must-not-be-named': 'Lord Voldemort',
    
    # Padres de Harry
    'james': 'James Potter',
    'james potter': 'James Potter',
    
    'lily': 'Lily Potter',
    'lily potter': 'Lily Potter',
    'lily evans': 'Lily Potter',
    },

    "LUGAR": [
        'hogwarts', 'diagon alley', 'knockturn alley', 'hogsmeade', 'quidditch pitch',
        'great hall', 'gryffindor tower', 'slytherin dungeons', 'hufflepuff',
        'ravenclaw tower', 'forbidden forest', 'whomping willow', 'chamber of secrets',
        'room of requirement', 'privet drive', 'little whinging', 'godrics hollow',
        'the burrow', 'grimmauld place', 'ministry of magic',
        'st mungos', 'azkaban', 'platform nine and three quarters', 'gringotts'
    ],
    "OBJETO_MAGICO": [
        'wand', 'elder wand', 'invisibility cloak', 'philosophers stone', 'sorcerers stone',
        'time turner', 'pensieve', 'sorting hat', 'marauders map', 'horcrux',
        'resurrection stone', 'deathly hallows', 'nimbus 2000', 'firebolt',
        'remembrall', 'howler', 'floo powder', 'portkey', 'mirror of erised', 'put-outer'
    ],
    "HECHIZO": [
        'expelliarmus', 'avada kedavra', 'crucio', 'imperio', 'expecto patronum',
        'alohomora', 'wingardium leviosa', 'accio', 'stupefy', 'protego',
        'sectumsempra', 'diffindo', 'incendio', 'aguamenti', 'lumos', 'nox'
    ]
}

PALABRAS_EXCLUIDAS= {
    'n’t', 'funny', 'nasty', 'harvey', 'harold', 'jim', 'ted',
    'muggle', 'muggles', 'witch', 'witches', 'wizard', 'wizards',
    'mr', 'mrs', 'ms', 'dear', 'sir', 'madam', 'bonfire night','yeh',
    'don', 'ter', 'an', 'er', 'o', 'a', 'i', 
    'jumped', 'people', 'said', 'looked', 'asked',
    'outta', 'kinda', 'gonna', 'wanna', 'gotta',
    'supreme', 'international', 'mugwump'
}

PATRONES_INVALIDOS = [
    r'^(harry|hermione|ron|dumbledore)\s+(jumped|said|looked|asked|walked|ran|thought)',
    r'^(yeh|ter|an|er)\s+',
    r'\s+(yeh|ter|an|er)$',
]


In [9]:

def es_entidad_valida(texto, palabras_excluidas, patrones_invalidos):
    if len(texto) < 2:
        return False
   
    palabras = texto.split()
    if any(palabra in palabras_excluidas for palabra in palabras):
        return False
    
    for patron in patrones_invalidos:
        if re.search(patron, texto):
            return False
    
    return True
def mapear_categoria_spacy(label_spacy):
    #Mapea etiquetas de spaCy a nuestras categorías
    mapeo = {
        'PERSON': 'PERSONAJE',
        'PER': 'PERSONAJE',
        'LOC': 'LUGAR',
        'GPE': 'LUGAR',
        'ORG': 'ORGANIZACION',
        'PRODUCT': 'OBJETO_MAGICO',
        'WORK_OF_ART': 'OBJETO_MAGICO'
    }
    return mapeo.get(label_spacy)


def normalizar_entidad(texto, categoria, entidades_dict):  
    texto_lower = texto.lower().strip()
    
    if categoria == "PERSONAJE" and "PERSONAJES" in entidades_dict:
        # Buscar coincidencia exacta primero
        if texto_lower in entidades_dict["PERSONAJES"]:
            return entidades_dict["PERSONAJES"][texto_lower]
        # Buscar coincidencias parciales o variantes
        for clave, nombre_canonico in entidades_dict["PERSONAJES"].items():
            # Si el texto contiene la clave o viceversa
            if clave in texto_lower or texto_lower in clave:
                return nombre_canonico
            
            # Verificar si es parte del nombre completo
            palabras_texto = set(texto_lower.split())
            palabras_clave = set(clave.split())
            if palabras_texto & palabras_clave:  # Si hay intersección
                return nombre_canonico
    
    elif categoria in ["LUGAR", "OBJETO_MAGICO", "HECHIZO"]:
        # Para otras categorías, buscar en las listas correspondientes
        categoria_plural = categoria + "S" if categoria != "HECHIZO" else categoria
        if categoria_plural in entidades_dict and isinstance(entidades_dict[categoria_plural], list):
            for entidad_conocida in entidades_dict[categoria_plural]:
                if texto_lower == entidad_conocida.lower():
                    return entidad_conocida.title()
    
    # Si no se encuentra normalización, devolver None para filtrar
    return None


def buscar_entidad_en_texto(entidad, texto):
    # Usar regex para buscar coincidencias con límites de palabra
    patron = r'\b' + re.escape(entidad) + r'\b'
    return bool(re.search(patron, texto, re.IGNORECASE))


# Función auxiliar para limpiar el resultado final
def limpiar_entidades_duplicadas(entidades_dict):
    entidades_limpias = {}
    
    for categoria, lista_entidades in entidades_dict.items():
        entidades_unicas = set(lista_entidades)
        
        # Para personajes, eliminar versiones parciales si existe la completa
        if categoria == "PERSONAJE":
            entidades_filtradas = set()
            for entidad in entidades_unicas:
                # Verificar si esta entidad es parte de otra más completa
                es_parcial = False
                for otra_entidad in entidades_unicas:
                    if entidad != otra_entidad and entidad in otra_entidad:
                        es_parcial = True
                        break
                
                if not es_parcial:
                    entidades_filtradas.add(entidad)
            
            entidades_unicas = entidades_filtradas
        
        entidades_limpias[categoria] = sorted(list(entidades_unicas))
    
    return entidades_limpias

In [10]:
"""
    Extrae entidades del documento y las normaliza a sus formas canónicas.
    
    Args:
        doc: Documento procesado por spaCy
        entidades_dict: Diccionario con las entidades conocidas
        palabras_excluidas: Conjunto de palabras a excluir
        patrones_invalidos: Lista de patrones regex a filtrar
        
    Returns:
        dict: Diccionario con entidades normalizadas por categoría
    """
def extraer_entidades(doc, entidades_dict, palabras_excluidas, patrones_invalidos):

    entidades_encontradas = defaultdict(set)
    
    # Procesar entidades detectadas por spaCy
    for ent in doc.ents:
        texto_entidad = ent.text.lower().strip()
        
        # Filtrar entidades inválidas
        if not es_entidad_valida(texto_entidad, palabras_excluidas, patrones_invalidos):
            continue
            
        # Mapear tipo de entidad spaCy a nuestras categorías
        categoria = mapear_categoria_spacy(ent.label_)
        
        if categoria:
            # Normalizar la entidad si es posible
            entidad_normalizada = normalizar_entidad(texto_entidad, categoria, entidades_dict)
            if entidad_normalizada:
                entidades_encontradas[categoria].add(entidad_normalizada)
    
    # Buscar entidades conocidas en el texto (para capturar las que spaCy no detectó)
    texto_lower = doc.text.lower()
    
    # Buscar personajes
    if "PERSONAJES" in entidades_dict:
        for clave, nombre_canonico in entidades_dict["PERSONAJES"].items():
            if buscar_entidad_en_texto(clave, texto_lower):
                entidades_encontradas["PERSONAJE"].add(nombre_canonico)
    
    # Buscar otras categorías (lugares, objetos mágicos, hechizos)
    for categoria, lista_entidades in entidades_dict.items():
        if categoria != "PERSONAJES" and isinstance(lista_entidades, list):
            categoria_singular = categoria.rstrip('S')  # Convertir plural a singular
            for entidad in lista_entidades:
                if buscar_entidad_en_texto(entidad, texto_lower):
                    entidades_encontradas[categoria_singular].add(entidad.title())
    
    # Convertir sets a listas ordenadas
    return {cat: sorted(list(ents)) for cat, ents in entidades_encontradas.items()}

In [18]:
print("=== PRUEBA DE EXTRACCIÓN DE ENTIDADES POR ÍNDICE ===\n")

# Tomar una muestra por índice para probar
indices_prueba = [15, 99, 120]
muestra_capitulos = df_libros.iloc[indices_prueba]

entidades_por_capitulo = []

print(f"Procesando capítulos en los índices: {indices_prueba}\n")

for idx, row in tqdm(muestra_capitulos.iterrows(), total=len(muestra_capitulos), desc="Procesando capítulos de prueba"):
    # Procesar el texto con spaCy
    doc = nlp(row['texto'][:100000])  # Limitar a 10k caracteres para velocidad
    
    # Extraer entidades
    entidades = extraer_entidades(doc, ENTIDADES_HP, PALABRAS_EXCLUIDAS, PATRONES_INVALIDOS)
    entidades_limpias = limpiar_entidades_duplicadas(entidades)
    
    entidades_por_capitulo.append({
        'libro': row['libro'],
        'capitulo': row['capitulo_num'],
        'titulo': row['titulo_capitulo'],
        'entidades': entidades_limpias
    })

# Mostrar resultados
for resultado in entidades_por_capitulo:
    print(f"\n📖 {resultado['libro']} - Cap {resultado['capitulo']}: {resultado['titulo']}")
    for categoria, entidades in resultado['entidades'].items():
        if entidades:  
            print(f"  {categoria}: {', '.join(entidades)}")

=== PRUEBA DE EXTRACCIÓN DE ENTIDADES POR ÍNDICE ===

Procesando capítulos en los índices: [15, 99, 120]



Procesando capítulos de prueba:   0%|          | 0/3 [00:00<?, ?it/s]


📖 01 Harry Potter and the Sorcerers Stone - Cap 16: THROUGH THE TRAPDOOR
  PERSONAJE: Albus Dumbledore, Harry Potter, Hermione Granger, Lord Voldemort, Minerva McGonagall, Ron Weasley, Rubeus Hagrid, Severus Snape
  LUGAR: Gringotts, Hogwarts, Ministry Of Magic
  OBJETO_MAGICO: Invisibility Cloak, Wand
  HECHIZO: Alohomora

📖 05 Harry Potter and the Order of the Phoenix - Cap 7: The Ministry of Magic
  PERSONAJE: Albus Dumbledore, Harry Potter, Ron Weasley, Severus Snape
  LUGAR: Hogwarts, Ministry Of Magic
  OBJETO_MAGICO: Portkey, Wand

📖 05 Harry Potter and the Order of the Phoenix - Cap 28: Snape's Worst Memory
  PERSONAJE: Albus Dumbledore, Harry Potter, Hermione Granger, James Potter, Lily Potter, Minerva McGonagall, Ron Weasley, Rubeus Hagrid, Severus Snape
  LUGAR: Great Hall, Gryffindor Tower, Hogwarts, Hufflepuff, Ministry Of Magic
  OBJETO_MAGICO: Firebolt, Pensieve, Wand
  HECHIZO: Accio, Expelliarmus, Stupefy


In [17]:
print("EXTRACCIÓN DE ENTIDADES DE TODO EL DATASET \n")

entidades_por_capitulo = []
for idx, row in tqdm(df_libros.iterrows(), total=len(df_libros), desc="Procesando todos los capítulos"):
    # Procesar el texto completo con spaCy (sin limitar caracteres)
    doc = nlp(row['texto'])

    entidades = extraer_entidades(doc, ENTIDADES_HP, PALABRAS_EXCLUIDAS, PATRONES_INVALIDOS)
    entidades_limpias = limpiar_entidades_duplicadas(entidades)
    entidades_por_capitulo.append({
        'libro': row['libro'],
        'capitulo': row['capitulo_num'],
        'titulo': row['titulo_capitulo'],
        'entidades': entidades_limpias
    })

df_entidades = pd.DataFrame(entidades_por_capitulo)

print("\n EXTRACCIÓN COMPLETA ")
print(f"Se procesaron {len(df_entidades)} capítulos.")
print("Vista previa del DataFrame con entidades:")
df_entidades.head()



EXTRACCIÓN DE ENTIDADES DE TODO EL DATASET 



Procesando todos los capítulos:   0%|          | 0/197 [00:00<?, ?it/s]


 EXTRACCIÓN COMPLETA 
Se procesaron 197 capítulos.
Vista previa del DataFrame con entidades:


,libro,capitulo,titulo,entidades
0,01 Harry Potter and the Sorcerers Stone,1,THE BOY WHO LIVED,"{'PERSONAJE': ['Albus Dumbledore', 'Dudley Dur..."
1,01 Harry Potter and the Sorcerers Stone,2,THE VANISHING GLASS,"{'PERSONAJE': ['Dudley Dursley', 'Harry Potter..."
2,01 Harry Potter and the Sorcerers Stone,3,LETTERS FROM NO ONE,"{'PERSONAJE': ['Dudley Dursley', 'Harry Potter..."
3,01 Harry Potter and the Sorcerers Stone,4,THE KEEPER OF THE KEYS,"{'PERSONAJE': ['Albus Dumbledore', 'Dudley Dur..."
4,01 Harry Potter and the Sorcerers Stone,5,DIAGON ALLEY,"{'PERSONAJE': ['Albus Dumbledore', 'Dudley Dur..."
